In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, confusion_matrix, make_scorer
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.svm import SVC
import matplotlib.pyplot as plt
import seaborn as sns
from sentence_transformers import SentenceTransformer
from scipy.spatial.distance import cosine
import warnings
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'imblearn'

In [ ]:
def load_data(file_path):
    """
    Load the dataset containing summaries and captions
    """
    # Assuming the data is in CSV format with columns: summary, caption, category
    df = pd.read_csv("manual_annotations.csv")
    print(f"Successfully loaded {len(df)} rows")
    return df

In [ ]:
# Feature engineering functions
def extract_features(df):
    """
    Extract features from summary and caption pairs
    """
    print("Extracting features...")
    
    # Load pre-trained sentence transformer model
    model = SentenceTransformer('all-MiniLM-L6-v2')
    
    # Generate embeddings for summaries and captions
    summary_embeddings = model.encode(df['Summary'].tolist())
    caption_embeddings = model.encode(df['Caption'].tolist())
    
    # Calculate cosine similarity between summary and caption
    cosine_similarities = []
    for i in range(len(df)):
        similarity = 1 - cosine(summary_embeddings[i], caption_embeddings[i])
        cosine_similarities.append(similarity)
    
    # Create additional features
    features = pd.DataFrame({
        'cosine_similarity': cosine_similarities,
        'summary_length': df['Summary'].apply(len),
        'caption_length': df['Caption'].apply(len),
        'length_ratio': df['Summary'].apply(len) / df['Caption'].apply(len),
        'word_count_summary': df['Summary'].apply(lambda x: len(x.split())),
        'word_count_caption': df['Caption'].apply(lambda x: len(x.split())),
        'word_count_ratio': df['Summary'].apply(lambda x: len(x.split())) / df['Caption'].apply(lambda x: len(x.split())),
    })
    
    # You could add more features here
    return features, df['Category']

In [ ]:
def train_evaluate_model(X, y, n_splits=5):
    """
    Train and evaluate models using stratified k-fold cross-validation
    """
    # Initialize the stratified k-fold 
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    
    # Define the models to try
    models = {
        'Logistic Regression': LogisticRegression(max_iter=1000, class_weight='balanced'),
        'SVM': SVC(class_weight='balanced', probability=True),
        'Random Forest': RandomForestClassifier(class_weight='balanced', n_estimators=100)
    }
    
    # Metrics to evaluate
    scoring = {
        'accuracy': make_scorer(accuracy_score),
        'precision_macro': make_scorer(precision_score, average='macro'),
        'recall_macro': make_scorer(recall_score, average='macro'),
        'f1_macro': make_scorer(f1_score, average='macro')
    }
    
    results = {}
    
    for name, model in models.items():
        print(f"\nTraining and evaluating {name}...")
        
        # Apply SMOTE for handling class imbalance within a Pipeline
        # using imblearn's Pipeline to correctly integrate SMOTE
        from imblearn.pipeline import Pipeline # Import imblearn's Pipeline
        
        pipeline = Pipeline([
            ('scaler', StandardScaler()),  # Apply scaling first if necessary
            # Adjust k_neighbors for SMOTE to handle smaller class sizes
            ('smote', SMOTE(random_state=42, k_neighbors=min(5, len(np.unique(y)) - 1))), # Then apply SMOTE
            ('classifier', model)  
        ])
        
        # Cross-validation
        cv_results = cross_validate(pipeline, X, y, 
                                    cv=skf, 
                                    scoring=scoring, 
                                    return_estimator=True,
                                    return_train_score=True)
        
        # Store results
        results[name] = {
            'cv_results': cv_results,
            'accuracy': cv_results['test_accuracy'].mean(),
            'precision': cv_results['test_precision_macro'].mean(),
            'recall': cv_results['test_recall_macro'].mean(),
            'f1': cv_results['test_f1_macro'].mean()
        }
        
        print(f"  Accuracy: {results[name]['accuracy']:.4f}")
        print(f"  Precision (macro): {results[name]['precision']:.4f}")
        print(f"  Recall (macro): {results[name]['recall']:.4f}")
        print(f"  F1 Score (macro): {results[name]['f1']:.4f}")
    
    return results

In [ ]:
def select_best_model(results):
    """
    Select the best model based on F1 macro score
    """
    scores = {name: results[name]['f1'] for name in results}
    best_model = max(scores, key=scores.get)
    print(f"\nBest model: {best_model} with F1 macro score: {scores[best_model]:.4f}")
    return best_model, results[best_model]

In [ ]:
# Function for detailed evaluation of the best model
def detailed_evaluation(X, y, best_model_name, best_model_results):
    """
    Perform detailed evaluation of the best model
    """
    print("\nDetailed evaluation of the best model...")
    
    # Get the best estimator from cross-validation
    best_estimator = best_model_results['cv_results']['estimator'][0]
    
    # Initialize a new StratifiedKFold for evaluation
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=43)
    
    all_predictions = []
    all_true_labels = []
    
    for train_idx, test_idx in skf.split(X, y):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
        
        # Refit the pipeline on this fold
        best_estimator.fit(X_train, y_train)
        
        # Get predictions
        y_pred = best_estimator.predict(X_test)
        
        # Store for aggregation
        all_predictions.extend(y_pred)
        all_true_labels.extend(y_test)
    
    # Overall classification report
    print("\nClassification Report:")
    print(classification_report(all_true_labels, all_predictions))
    
    # Confusion Matrix
    cm = confusion_matrix(all_true_labels, all_predictions)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=np.unique(y),
                yticklabels=np.unique(y))
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title(f'Confusion Matrix - {best_model_name}')
    plt.show()
    
    return {
        'classification_report': classification_report(all_true_labels, all_predictions, output_dict=True),
        'confusion_matrix': cm
    }

In [ ]:
def analyze_feature_importance(X, best_model_results):
    """
    Analyze feature importance for the best model (if applicable)
    """
    best_estimator = best_model_results['cv_results']['estimator'][0]
    
    # Check if the model supports feature importance
    if hasattr(best_estimator[-1], 'feature_importances_'):
        importances = best_estimator[-1].feature_importances_
        feature_names = X.columns
        
        # Create and plot feature importance
        importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': importances})
        importance_df = importance_df.sort_values('Importance', ascending=False)
        
        plt.figure(figsize=(12, 8))
        sns.barplot(x='Importance', y='Feature', data=importance_df)
        plt.title('Feature Importance')
        plt.tight_layout()
        plt.show()
        
        return importance_df
    
    elif hasattr(best_estimator[-1], 'coef_'):
        # For models like logistic regression
        coefficients = best_estimator[-1].coef_
        feature_names = X.columns
        
        # If multiclass, take average absolute coefficient
        if len(coefficients.shape) > 1:
            importances = np.mean(np.abs(coefficients), axis=0)
        else:
            importances = np.abs(coefficients)
            
        # Create and plot feature importance
        importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': importances})
        importance_df = importance_df.sort_values('Importance', ascending=False)
        
        plt.figure(figsize=(12, 8))
        sns.barplot(x='Importance', y='Feature', data=importance_df)
        plt.title('Feature Importance')
        plt.tight_layout()
        plt.show()
        
        return importance_df
    
    else:
        print("Feature importance not available for this model type")
        return None